# Ensemble methods. Exercises


In this section we have only two exercise:

1. Find the best three classifier in the stacking method using the classifiers from scikit-learn package.

2. Build arcing arc-x4 method. 

In [3]:
from sklearn.datasets import load_iris

iris = load_iris()

data_set = iris.data[0:len(iris.target)-20,:]
labels = iris.target[0:len(iris.target)-20]
unique_labels = np.unique(iris.target)

test_data_set = iris.data[-20:,:]
test_labels = iris.target[-20:]

%store data_set
%store labels
%store test_data_set
%store test_labels
%store unique_labels

Stored 'data_set' (ndarray)
Stored 'labels' (ndarray)
Stored 'test_data_set' (ndarray)
Stored 'test_labels' (ndarray)
Stored 'unique_labels' (ndarray)


## Exercise 1: Find the best three classifier in the stacking method

Please use the following classifiers:

* Linear regression,
* Nearest Neighbors,
* Linear SVM,
* Decision Tree,
* Naive Bayes,
* QDA.

In [7]:
import numpy as np
from sklearn.metrics import accuracy_score

from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.ensemble import StackingClassifier

In [12]:
def build_classifiers(X, y):
    classifiers = []
    
    model_lr = LinearRegression()
    model_lr.fit(X, y)
    classifiers.append(model_lr)
    
    model_kn = KNeighborsClassifier()
    model_kn.fit(X, y)
    classifiers.append(model_kn)
    
    model_svm = SVC()
    model_svm.fit(X, y)
    classifiers.append(model_svm)
    
    model_dt = DecisionTreeClassifier()
    model_dt.fit(X, y)
    classifiers.append(model_dt)
    
    model_nb = GaussianNB()
    model_nb.fit(X, y)
    classifiers.append(model_nb)
    
    model_qda = QuadraticDiscriminantAnalysis()
    model_qda.fit(X, y)
    classifiers.append(model_qda)

    return classifiers

In [29]:
def build_stacked_classifier(classifiers):
    output = []
    for classifier in classifiers:
        output.append(classifier.predict(data_set))
    output = np.array(output).reshape((130,6))
    
    # stacked classifier part:
    stacked_classifier = DecisionTreeClassifier()
    stacked_classifier.fit(output.reshape((130,6)), labels.reshape((130,)))
    test_set = []
    
    for classifier in classifiers:
        test_set.append(classifier.predict(test_data_set))
        
    test_set = np.array(test_set).reshape((len(test_set[0]),6))
    predicted = stacked_classifier.predict(test_set)
    
    return predicted

In [30]:
classifiers = build_classifiers(data_set, labels)
predicted = build_stacked_classifier(classifiers)
accuracy = accuracy_score(test_labels, predicted)
print(accuracy)

0.55


## Exercise 2: 

Use the boosting method and change the code to fullfilt the following requirements:

* the weights should be calculated as:
$w_{n}^{(t+1)}=\frac{1+ I(y_{n}\neq h_{t}(x_{n})}{\sum_{i=1}^{N}1+I(y_{n}\neq h_{t}(x_{n})}$,
* the prediction is done with a voting method.

In [55]:
import numpy as np
from sklearn.tree import DecisionTreeClassifier

# prepare data set

def generate_data(sample_number, feature_number, label_number):
    data_set = np.random.random_sample((sample_number, feature_number))
    labels = np.random.choice(label_number, sample_number)
    return data_set, labels

labels = 2
dimension = 2
test_set_size = 1000
train_set_size = 5000
train_set, train_labels = generate_data(train_set_size, dimension, labels)
test_set, test_labels = generate_data(test_set_size, dimension, labels)

# init weights
number_of_iterations = 10
weights = np.ones((test_set_size,)) / test_set_size


def train_model(classifier, weights):
    return classifier.fit(X=test_set, y=test_labels, sample_weight=weights)

def calculate_error(model):
    predicted = model.predict(test_set)
    I=calculate_accuracy_vector(predicted, test_labels)
    Z=np.sum(I)
    return (1+Z)/1.0

Fill the two functions below:

In [56]:


def calculate_accuracy_vector(predicted, labels):
    result = []
    for i in range(len(predicted)):
        if predicted[i] == labels[i]:
            result.append(0)
        else:
            result.append(1)
    return result

def set_new_weights(model):
    new_weights = []
    accuracy_vector = calculate_accuracy_vector(model.predict(test_set), test_labels)
    
    new_weights = np.add(np.ones(len(accuracy_vector)), accuracy_vector)
        
    return new_weights / np.sum(new_weights)

Train the classifier with the code below:

In [57]:
classifier = DecisionTreeClassifier(max_depth=1, random_state=1)
classifier.fit(X=train_set, y=train_labels)
alphas = []
classifiers = []
print(weights)
for iteration in range(number_of_iterations):
    model = train_model(classifier, weights)
    weights = set_new_weights(model)
    classifiers.append(model)

print(weights)


validate_x, validate_label = generate_data(1, dimension, labels)

[0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001
 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001
 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001
 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001
 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001
 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001
 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001
 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001
 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001
 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001
 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001
 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001
 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001
 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.

Set the validation data set:

In [58]:
validate_x, validate_label = generate_data(1, dimension, labels)

Fill the prediction code:

In [64]:
def get_prediction(x):
    output = []
    predicted = []
    
    for classifier in classifiers:
        output.append(classifier.predict(x))
        
    output = np.array(output)
    
    for i in range(len(x)):
        classified = output[:, i]
        counts = np.bincount(classified)
        predicted.append(np.argmax(counts))
        
    return predicted

Test it:

In [65]:
prediction = get_prediction(validate_x)[0]

print(prediction)

1
